#### 1. Importing Libraries

In [1]:
import sys
import requests
import bs4
import json
import re
import time
from nltk.tokenize import sent_tokenize
import re

#### 2. Extracting WikiPages for all English Labelled Monuments

In [2]:
person_list = []
for i in range(1,25):
    with open('./Data/Wikidata_JSON_businessperson/en_businessperson_dump_part'+str(i)+'.json') as f:
        new_list = json.load(f)
        person_list = person_list + new_list

In [10]:
t0 = time.time()
articles = {}
count_persons = 0
curr_count_persons = 0
for person in person_list:
    res = requests.get('https://en.wikipedia.org/wiki/' + person['labels']['en']['value'])
    # print(res.text)
    if res.status_code != 404:
        wiki = bs4.BeautifulSoup(res.text,"html.parser")
        elems = wiki.select('p')
        article = []
        for i in range(len(elems)):
            text = elems[i].getText().replace('\n', '').lower()
            text = text.replace('\ufeff', ' ')
            text = re.sub('\[\d+\]', '', text.strip())
            article.extend(sent_tokenize(text))
    
        if '' in article:
            article.remove('')
        
        for i in range(0,len(article)):
            article[i] = article[i].strip()
            
        articles[person['id']] = article
    
    count_persons = count_persons + 1
    
    if count_persons%200 == 0:
        partition_num = str(count_persons/200).split('.')[0]
        with open('./Data/English_Wikipages_businessperson/en_articles_part'+ partition_num + '.json', 'w') as fout:
            json.dump(articles, fout)
        
        articles = {}
        
        print("Checkpoint %d reached, JSON dumps saved |" % (count_persons/200), end = ' ')
        print("Time Elapsed:", end = ' ')
        print(time.time()-t0)

t1 = time.time()
total = t1-t0

Checkpoint 1 reached, JSON dumps saved | Time Elapsed: 71.89372253417969
Checkpoint 2 reached, JSON dumps saved | Time Elapsed: 152.0993046760559
Checkpoint 3 reached, JSON dumps saved | Time Elapsed: 239.07315397262573
Checkpoint 4 reached, JSON dumps saved | Time Elapsed: 320.444961309433
Checkpoint 5 reached, JSON dumps saved | Time Elapsed: 400.30911898612976
Checkpoint 6 reached, JSON dumps saved | Time Elapsed: 474.4340031147003
Checkpoint 7 reached, JSON dumps saved | Time Elapsed: 556.4206521511078
Checkpoint 8 reached, JSON dumps saved | Time Elapsed: 635.4039008617401
Checkpoint 9 reached, JSON dumps saved | Time Elapsed: 719.8054065704346
Checkpoint 10 reached, JSON dumps saved | Time Elapsed: 802.2399859428406
Checkpoint 11 reached, JSON dumps saved | Time Elapsed: 882.3462934494019
Checkpoint 12 reached, JSON dumps saved | Time Elapsed: 963.8035509586334
Checkpoint 13 reached, JSON dumps saved | Time Elapsed: 1043.0440764427185
Checkpoint 14 reached, JSON dumps saved | Tim

In [11]:
print(total)
print(len(articles))

1975.035441160202
141


In [14]:
tot_len = 0
for i in range(1,23):
    with open('./Data/English_Wikipages_businessperson/en_articles_part'+str(i)+'.json') as f:
        tot_len += len(json.load(f))
tot_len

3718